## Libraries

In [ ]:
# import libraries
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

## Functions

### Data pre-processing

In [ ]:
def parse_tblout(file_path, e_value_threshold=None):
    """
    Parse a .tblout file and create a DataFrame with optional E-value filtering
    """
    # Define column names
    columns = [
        "target_name", "accession", "tlen", "query_name", "query_accession", "qlen",
        "E-value", "score", "bias", "#", "of", "c-Evalue", "i-Evalue",
        "domain_score", "domain_bias", "hmm_from", "hmm_to", "ali_from", "ali_to",
        "env_from", "env_to", "acc", "description_of_target"
    ]

    # read the file
    with open(file_path, "r") as f:
        lines = f.readlines()

    # extract data lines (skip header and dashed lines)
    data_lines = lines[3:]  # skip the header and separator lines
    data_lines = [line.strip() for line in data_lines if not line.startswith("#")]

    # parse the data into rows
    rows = [line.split() for line in data_lines]
    data_table = pd.DataFrame(rows, columns=columns)

    # convert numeric columns to proper types
    numeric_cols = ["E-value", "c-Evalue", "i-Evalue", "score", "domain_score"]
    data_table[numeric_cols] = data_table[numeric_cols].apply(pd.to_numeric, errors="coerce")

    # apply E-value filtering if threshold is provided
    if e_value_threshold is not None:
        data_table = data_table[data_table["E-value"] <= e_value_threshold]

    return data_table

In [ ]:
def create_binary_matrix(data_table, sequence_count):
    """
    Create a binary matrix with sequences as rows and targets as columns.

    Parameters:
    data_table: pd.DataFrame
        Table with "query_name" and "target_name" columns.
    sequence_count: int
        Total number of sequences (seq1 to seqN).

    Returns:
    pd.DataFrame
        Binary matrix with 1 for presence and 0 for absence of targets.
    """
    # create a binary matrix using crosstab
    binary_matrix = pd.crosstab(data_table["query_name"], data_table["target_name"])

    # ensure the values are binary (1/0)
    binary_matrix = binary_matrix.astype(bool).astype(int)

    # remove the names for rows and columns
    binary_matrix.index.name = None
    binary_matrix.columns.name = None

    # generate the full sequence list (seq1 to seqN)
    full_sequence_list = [f"seq{i}" for i in range(1, sequence_count + 1)]

    # reindex the binary matrix to include all sequences, filling missing rows with zeros
    binary_matrix = binary_matrix.reindex(full_sequence_list, fill_value=0)
    binary_matrix.index.name = "sequence"

    # remove columns where all values are 0
    binary_matrix = binary_matrix.loc[:, binary_matrix.any(axis=0)]

    return binary_matrix

### Model exploration

In [ ]:
def load_data(descriptor):
    """
    Loads feature data from a CSV file, sets the sequence ID as the index,
    and filters out features with zero variance.

    Parameters:
    descriptor: str
        The name of the feature descriptor, used to locate the CSV file.

    Returns:
    pd.DataFrame
        A df with sequences as rows and filtered features as columns.
    """
    file_path = os.path.join(input_path, f"{descriptor}.csv")
    x = pd.read_csv(file_path)
    x.set_index(x.columns[0], inplace=True)
    x.rename_axis("sequence", inplace=True)
    return x.loc[:, (x != 0).any(axis=0)]

In [ ]:
def plot_top_features(x, descriptor, top_n=10):
    """
    Plot and save histograms of the top features.

    Parameters:
    Descriptor: str
        name of the feature descriptor used for labeling the plot and saving the file.
    top_n: int, optional
        The number of top features to plot (default is 10).
    x: pd.DataFrame
        Feature data where rows are sequences and columns are features.

    Saves:
    A histogram plot of the top features in the `results_dir` folder.
    """
    # determine the actual number of features to plot
    available_features = min(top_n, x.shape[1])  # use top_n or total number of columns, whichever is smaller
    top_features = x.mean(axis=0).nlargest(available_features).index
    plt.figure(figsize=(12, 8))
    for feature in top_features:
        plt.hist(x[feature], bins=30, alpha=0.5, label=feature)

    plt.title(f"Top {available_features} Features Distribution for {descriptor}")
    plt.xlabel("Feature values")
    plt.ylabel("Frequency")
    plt.legend(loc="upper right")
    plt.savefig(os.path.join(results_dir, f"{descriptor}_top_features_distribution.png"))
    plt.close()

In [ ]:
def save_statistics(x, descriptor):
    """
    Calculate and save descriptive statistics for each feature.
    """
    stats = x.describe()
    stats.to_csv(os.path.join(results_dir, f"{descriptor}_stats.csv"))

In [ ]:
def calculate_and_plot_correlation(x, y, descriptor):
    """
    Calculate correlations of features with a target variable, save the results,
    and plot the sorted correlations.

    Parameters:
    x: pd.DataFrame
        Feature data with rows as samples and columns as features.
    y: pd.Series
        Target variable (e.g., infection status).
    descriptor: str
        Descriptor name for labeling and saving the outputs.

    Returns:
    pd.Series
        Sorted correlations of features with the target variable.
    """
    # combine feature data (x) and target variable (y) into a single df
    combined_df = pd.concat([x, y], axis=1)

    # calculate the correlation matrix
    correlation_matrix = combined_df.corr()

    # extract correlation with the target variable and remove the target itself
    correlation_with_target = correlation_matrix['infection'].drop('infection')

    # sort correlations in descending order
    sorted_correlations = correlation_with_target.sort_values(ascending=False)

    # save the sorted correlations to a CSV file
    sorted_correlations.to_csv(os.path.join(results_dir, f"{descriptor}_correlation.csv"))

    # plot the sorted correlations
    plt.figure(figsize=(12, 6))
    sns.barplot(x=sorted_correlations.values, y=sorted_correlations.index)
    plt.title(f'Correlation of {descriptor} Features with Infection Status')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel(f'{descriptor} Features')
    plt.axvline(0, color='gray', linestyle='--')

    # save and close the plot
    plt.savefig(os.path.join(results_dir, f"{descriptor}_correlation_plot.png"))
    plt.close()

    # return sorted correlations for potential further use
    return sorted_correlations

In [ ]:
def select_features_by_correlation(x, correlation_with_target, threshold=0):
    """
    Select features based on a correlation threshold if more than 100 features exist.

    Parameters:
    x: pd.DataFrame
        Feature data with rows as samples and columns as features.
    correlation_with_target: pd.Series
        Correlation values of features with the target variable.
    threshold: float, optional
        Minimum absolute correlation value to select features (default is 0).

    Returns:
    pd.DataFrame
        Filtered feature data containing only the selected features.
    """
    # add this moment is 0 as is not being use ## double check
    # check if the number of descriptors is greater than 100
    if len(correlation_with_target) > 100:
        # apply threshold if more than 100 descriptors
        selected_features = correlation_with_target[correlation_with_target.abs() > threshold].index.tolist()
    else:
        # select all features if 100 or fewer descriptors are available
        selected_features = correlation_with_target.index.tolist()

    return x[selected_features]

In [ ]:
def plot_pairplot(x_selected, y_selected, descriptor):
    """
    Create and save a pairplot for the selected features.
    """
    pairplot_data = pd.concat([x_selected, y_selected], axis=1)
    pairplot_data.rename(columns={'infection': 'Target'}, inplace=True)
    sns.pairplot(pairplot_data, hue='Target', diag_kind='kde', markers=["o", "s"])
    plt.savefig(os.path.join(results_dir, f"{descriptor}_pairplot.png"))
    plt.close()

In [ ]:
def train_and_evaluate_model(x_train, x_test, y_train, y_test, descriptor):
    """
    Train a Random Forest model, evaluate its performance, and save the results.

    Parameters:
    x_train: pd.DataFrame
        Training feature data.
    x_test: pd.DataFrame
        Test feature data.
    y_train: pd.Series or np.array
        Training target labels.
    y_test: pd.Series or np.array
        Test target labels.
    descriptor: str
        Descriptor name for labeling and saving the output file.

    Returns:
    RandomForestClassifier
        Trained Random Forest model.
    """
    model = RandomForestClassifier(random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # accuracy and classification report
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    with open(os.path.join(results_dir, f"{descriptor}_evaluation.txt"), "w") as f:
        f.write(f"Accuracy: {accuracy}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    return model

In [ ]:
def cross_validate_model(model, x_selected, y_selected, descriptor):
    """
    Perform 10-fold cross-validation on the model and save the results.
    """
    cv_scores = cross_val_score(model, x_selected, y_selected, cv=10)
    with open(os.path.join(results_dir, f"{descriptor}_cv_scores.txt"), "w") as f:
        f.write("Cross-Validation Scores:\n")
        f.write(", ".join(map(str, cv_scores)) + "\n")
        f.write(f"Mean Cross-Validation Score: {cv_scores.mean()}")

In [ ]:
def plot_feature_importance(model, x_selected, descriptor):
    """
    Plot and save feature importance for the trained model.
    """
    importance_df = pd.DataFrame({'Feature': x_selected.columns, 'Importance': model.feature_importances_})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Importance', y='Feature', data=importance_df)
    plt.title(f"Feature Importances for {descriptor}")
    plt.savefig(os.path.join(results_dir, f"{descriptor}_feature_importance.png"))
    plt.close()

## Data parsing

In [ ]:
# define directory
os.chdir("/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project")

# parse vog and vfam txt
vog_table = parse_tblout("data/phmms/vog/vog_matches_domtblout.txt", e_value_threshold=0.01)
vfam_table = parse_tblout("data/phmms/vfam/vfam_matches_domtblout.txt", e_value_threshold=0.01)

# create binary matrices
vog_binary_matrix = create_binary_matrix(vog_table, sequence_count=3936)
vfam_binary_matrix = create_binary_matrix(vfam_table, sequence_count=3936)

# concatenate vog and vfam
allphmms_binary_matrix = pd.concat([vog_binary_matrix, vfam_binary_matrix], axis=1)

# replace NaN values with 0 to ensure binary data
allphmms_binary_matrix = allphmms_binary_matrix.fillna(0).astype(int)

# save tables to CSV file
vog_binary_matrix.to_csv("features/pro_features/phmms/vog/vog_table.csv", index=True)
vfam_binary_matrix.to_csv("features/pro_features/phmms/vfam/vfam_table.csv", index=True)
allphmms_binary_matrix.to_csv("features/pro_features/phmms/allphmms_table.csv", index=True)

Display and print some stats for the PHMMs

In [ ]:
# display vog and vfam dfs
print("VOG table:")
print(vog_table.shape)
print(vog_table.head())

# print("\nVFAM table:")
print(vfam_table.shape)
print(vfam_table.head())


# display binary matrices
print("\nVOG Binary Matrix:")
print(vog_binary_matrix.shape)
print(vog_binary_matrix.head())

print("\nVFAM Binary Matrix:")
print(vfam_binary_matrix.shape)
print(vfam_binary_matrix.head())

print("\nPHMMs Binary Matrix:")
print(allphmms_binary_matrix.shape)
print(allphmms_binary_matrix.head())

VOG table:
(560677, 23)
  target_name accession  tlen query_name query_accession  qlen        E-value  \
0    VOG23377         -   968       seq1               -  1629  6.900000e-225   
1    VOG69164         -  1254       seq1               -  1629   2.900000e-82   
2    VOG69164         -  1254       seq1               -  1629   2.900000e-82   
3    VOG03312         -   574       seq1               -  1629   9.500000e-76   
4    VOG35419         -  1800       seq1               -  1629   3.400000e-58   

   score bias  #  ... domain_score  domain_bias  hmm_from  hmm_to ali_from  \
0  751.1  0.0  1  ...        750.7          0.0        56     943       42   
1  278.4  0.0  1  ...         36.1          0.1       213     433       60   
2  278.4  0.0  2  ...        233.7          0.0       578    1084      362   
3  258.0  9.3  1  ...        256.9          9.3        29     470     1170   
4  198.5  7.0  1  ...        197.9          7.0      1311    1693     1209   

  ali_to env_from en

## Train, test and evaluate

In [ ]:
y = pd.read_csv('/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/data/y.csv', na_values=['?'])
y.set_index('sequence', inplace=True)

# define paths
input_path = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/features/pro_features"
results_dir = "/Users/lizsteny/Documents/Academia/MSc Bioinformatics/Project/bif_project/results/pro_results"
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# main loop to process each descriptor file
descriptors = [
"vog_table", "vfam_table", "allphmms_table"
]

for descriptor in descriptors:
    # load data
    x = load_data(descriptor)

    # plot top features distribution
    plot_top_features(x, descriptor)

    # save descriptive statistics
    save_statistics(x, descriptor)

    # calculate and save correlation
    correlation_with_target = calculate_and_plot_correlation(x, y['infection'], descriptor)

    # select features based on correlation threshold
    x_selected = select_features_by_correlation(x, correlation_with_target)
    y_selected = y['infection']

    # plot pairplot
    # plot_pairplot(x_selected, y_selected, descriptor)

    # train-test split
    x_train, x_test, y_train, y_test = train_test_split(x_selected, y_selected, test_size=0.2, random_state=42)

    # train model and evaluate
    model = train_and_evaluate_model(x_train, x_test, y_train, y_test, descriptor)

    # cross-validation
    cross_validate_model(model, x_selected, y_selected, descriptor)

    # plot feature importances
    plot_feature_importance(model, x_selected, descriptor)

    print(f"Processed and saved results for {descriptor}")

In [ ]:
data = []

# loop through each descriptor and extract evaluation and CV scores
for descriptor in descriptors:
    # define file paths for evaluation and CV scores
    evaluation_file = os.path.join(results_dir, f"{descriptor}_evaluation.txt")
    cv_scores_file = os.path.join(results_dir, f"{descriptor}_cv_scores.txt")

    # initialize variables to store scores
    accuracy = None
    classification_report = None
    cv_scores = []
    mean_cv_score = None

    # read and parse the evaluation file if it exists
    if os.path.exists(evaluation_file):
        with open(evaluation_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith("Accuracy:"):
                    accuracy = float(line.split(":")[1].strip())
                elif line.startswith("Classification Report:"):
                    classification_report = "".join(lines[lines.index(line)+1:]).strip()  # collect the full classification report
    else:
        print(f"Warning: Evaluation file for {descriptor} not found.")

    # read and parse the CV scores file if it exists
    if os.path.exists(cv_scores_file):
        with open(cv_scores_file, 'r') as f:
            lines = f.readlines()
            # extract CV scores and mean CV score
            cv_scores = [float(score) for score in re.findall(r"\d+\.\d+", lines[1])]  # assuming scores are on the second line
            mean_cv_score = float(re.search(r"Mean Cross-Validation Score: (\d+\.\d+)", lines[-1]).group(1))
    else:
        print(f"Warning: CV scores file for {descriptor} not found.")

    # add a row to the data list
    data.append({
        "Method": descriptor,
        "Accuracy": accuracy,
        "Mean CV Score": mean_cv_score,
        "CV Scores": cv_scores,
        "Classification Report": classification_report
    })

# create a df from the collected data
df = pd.DataFrame(data)

# save the df to a CSV file
df.to_csv(os.path.join(results_dir, "summary/combined_evaluation_cv_scores.csv"), index=False)
print("Combined evaluation and CV scores saved to combined_evaluation_cv_scores.csv")